In [13]:
! pip install spacy


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
! python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     -------------------- ------------------- 6.6/12.8 MB 33.6 MB/s eta 0:00:01
     --------------------------------------- 12.8/12.8 MB 32.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
import pandas as pd

In [19]:
df=pd.read_csv("pronoun_testcases.csv")

In [20]:
print(df.head())


                   input_text target_gender              expected_output
0  He is going to the market.        female  She is going to the market.
1   His book is on the table.        female    Her book is on the table.
2        I saw him yesterday.        female         I saw her yesterday.
3            He hurt himself.        female            She hurt herself.
4    I called him last night.        female     I called her last night.


In [24]:
import spacy

# Load the English model
nlp = spacy.load("en_core_web_sm")

def exchange_pronouns(text, target_gender):
    doc = nlp(text)
    
    replacements = []
    
    for token in doc:
        original_text = token.text
        replacement = None
        
        pos = token.pos_
        dep = token.dep_
        
        if target_gender.lower() == 'female':
            if original_text.lower() == 'he':
                replacement = 'She' if original_text[0].isupper() else 'she'
            elif original_text.lower() == 'him':
                replacement = 'Her' if original_text[0].isupper() else 'her'
            elif original_text.lower() == 'his':
                replacement = 'Her' if original_text[0].isupper() else 'her'
            elif original_text.lower() == 'himself':
                replacement = 'Herself' if original_text[0].isupper() else 'herself'
                
        elif target_gender.lower() == 'male':
            if original_text.lower() == 'she':
                replacement = 'He' if original_text[0].isupper() else 'he'
            elif original_text.lower() == 'her':
                if dep == 'poss' or pos == 'PRON' and dep == 'nmod':
                    replacement = 'His' if original_text[0].isupper() else 'his'
                else:
                    replacement = 'Him' if original_text[0].isupper() else 'him'
            elif original_text.lower() == 'hers':
                replacement = 'His' if original_text[0].isupper() else 'his'
            elif original_text.lower() == 'herself':
                replacement = 'Himself' if original_text[0].isupper() else 'himself'
        
        if replacement:
            replacements.append((token.idx, token.idx + len(token.text), replacement))
    
    result = text
    for start, end, replacement in reversed(replacements):
        result = result[:start] + replacement + result[end:]
    
    return result
doc = nlp("She gave him her book.")
for token in doc:
    print(f"Token: '{token.text}', POS: {token.pos_}, Dep: {token.dep_}, Head: {token.head.text}")
print()

Token: 'She', POS: PRON, Dep: nsubj, Head: gave
Token: 'gave', POS: VERB, Dep: ROOT, Head: gave
Token: 'him', POS: PRON, Dep: dative, Head: gave
Token: 'her', POS: PRON, Dep: poss, Head: book
Token: 'book', POS: NOUN, Dep: dobj, Head: gave
Token: '.', POS: PUNCT, Dep: punct, Head: gave



In [25]:
df['predicted_output'] = df.apply(lambda row: exchange_pronouns(row['input_text'], row['target_gender']), axis=1)

correct = 0
total = len(df)

print(f"Total test cases: {total}")
print()

for i, row in df.iterrows():
    is_correct = row['predicted_output'] == row['expected_output']
    if is_correct:
        correct += 1
    else:
        print(f"  Test {i+1} FAILED:")
        print(f"   Input: '{row['input_text']}'")
        print(f"   Target: {row['target_gender']}")
        print(f"   Expected: '{row['expected_output']}'")
        print(f"   Got: '{row['predicted_output']}'")
        print()

print(f" Accuracy: {correct}/{total} = {correct/total:.2%}")

print("\n Sample successful transformations:")
successful_cases = df[df['predicted_output'] == df['expected_output']].head(5)
for i, row in successful_cases.iterrows():
    print(f"  '{row['input_text']}' : '{row['predicted_output']}'")

# Linguistic analysis for a random sample
import random
random_idx = random.randint(0, len(df)-1)
sample_row = df.iloc[random_idx]

print(f"\n🔍 Linguistic Analysis for Random Sample (Row {random_idx + 1}):")
print(f"Input: '{sample_row['input_text']}'")
print(f"Output: '{sample_row['predicted_output']}'")
print("\nToken Analysis:")

doc = nlp(sample_row['input_text'])
for token in doc:
    print(f"Token: '{token.text}', POS: {token.pos_}, Dep: {token.dep_}, Head: {token.head.text}")
print()

Total test cases: 26

 Accuracy: 26/26 = 100.00%

 Sample successful transformations:
  'He is going to the market.' : 'She is going to the market.'
  'His book is on the table.' : 'Her book is on the table.'
  'I saw him yesterday.' : 'I saw her yesterday.'
  'He hurt himself.' : 'She hurt herself.'
  'I called him last night.' : 'I called her last night.'

🔍 Linguistic Analysis for Random Sample (Row 23):
Input: 'She brought her laptop.'
Output: 'He brought his laptop.'

Token Analysis:
Token: 'She', POS: PRON, Dep: nsubj, Head: brought
Token: 'brought', POS: VERB, Dep: ROOT, Head: brought
Token: 'her', POS: PRON, Dep: poss, Head: laptop
Token: 'laptop', POS: NOUN, Dep: dobj, Head: brought
Token: '.', POS: PUNCT, Dep: punct, Head: brought

